In [ ]:
from pathlib import Path
import sys
import pypsa
import pandas as pd


def find_repo_root(max_up=6):
    p = Path.cwd().resolve()
    for _ in range(max_up):
        if (p / 'README.md').exists() or (p / '.git').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return Path.cwd().resolve()

repo_root = find_repo_root()
src_path = repo_root / 'src/'
if str(src_path) not in sys.path:
    sys.path.insert(1, str(src_path))
print(f"Using src path: {src_path}")
print(f"Repository root: {repo_root}")

import pypsa_simplified as ps

src_path = repo_root / 'scripts/'
if str(src_path) not in sys.path:
    sys.path.insert(1, str(src_path))

import geometry as geom

def ifjoin(n: pypsa.Network) -> bool:
    """Helper function to conditionally join network buses."""
    return "[join]" in str(n.name)

def iffloat(n: pypsa.Network) -> bool:
    """Helper function to conditionally join network buses."""
    return "[float]" in str(n.name)

import network_clust as netclust

JOIN = True
FLOAT_ = True

In [ ]:
# Load a clustered model

n_clusters_target = 200
strategy_name = 'gurobi'

cluster_path = repo_root / "data" / "networks" / "clustered" / f"C+_sEEN{'_join' if JOIN else ''}{'_f' if FLOAT_ else ''}_cl{n_clusters_target}_{strategy_name}.nc"
n = pypsa.Network(simplified_path)
